# Lab 3

In [1]:
import numpy as np
import random
from enum import Enum

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [2]:
def generate_city_costs(cities_count=100, low=10.0, high=90.0):
    costs = np.random.uniform(low, high, size=(cities_count, cities_count))
    np.fill_diagonal(costs, 0.0)
    return costs

def route_cost(route, costs):
    return costs[route, np.roll(route, -1)].sum()

In [3]:
def mutate(route, rng, k):
    child = route.copy()
    idx = rng.choice(len(route), size=k, replace=False)
    if k == 2:
        a, b = idx
        child[a], child[b] = child[b], child[a]
    elif k == 3:
        a, b, c = idx
        child[a], child[b], child[c] = child[b], child[c], child[a]
    else:
        raise NotImplementedError(f"Strategy k={k} is not supported.")
    return child

def evolutionary_tsp(costs, max_offspring=10_000, stall_limit=2_000, seed=None):
    rng = np.random.default_rng(seed)
    parent = rng.permutation(costs.shape[0])
    parent_cost = route_cost(parent, costs)

    stall = 0
    for t in range(max_offspring):
        # If too long without improvement, try stronger mutation
        k = 2 if (stall < stall_limit) else 3

        child = mutate(parent, rng, k=k)
        child_cost = route_cost(child, costs)

        if child_cost < parent_cost:
            parent, parent_cost = child, child_cost
            stall = 0
        else:
            stall += 1
        if stall >= 2 * stall_limit:
            break

    return parent, parent_cost

def run_multiple_times(costs, runs=10, seed=123):
    best_route, best_cost = None, float("inf")
    for i in range(runs):
        route, cost = evolutionary_tsp(costs, seed=seed + i)
        if cost < best_cost:
            best_route, best_cost = route, cost
    return best_route, best_cost

In [4]:
costs = generate_city_costs(cities_count=100)
best_route, best_cost = run_multiple_times(costs, runs=10)
print("Best cost:", best_cost)
print("Best route (first 10 cities):", best_route[:10])


Best cost: 1820.2902196603723
Best route (first 10 cities): [81 42 95 92 46 11 29 34  2  5]


In [5]:
def ortools_tsp(costs):
    n = costs.shape[0]

    manager = pywrapcp.RoutingIndexManager(n, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_idx, to_idx):
        i = manager.IndexToNode(from_idx)
        j = manager.IndexToNode(to_idx)
        return int(costs[i, j] * 1000)

    transit_callback = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback)

    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_params.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_params.time_limit.seconds = 5

    solution = routing.SolveWithParameters(search_params)
    if solution is None:
        return None, None

    index = routing.Start(0)
    route = []
    cost = 0
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        next_index = solution.Value(routing.NextVar(index))
        cost += routing.GetArcCostForVehicle(index, next_index, 0)
        index = next_index

    return np.array(route), cost / 1000


In [6]:
ort_route, ort_cost = ortools_tsp(costs)

print("Custom cost:", best_cost)
print("OR-Tools cost:", ort_cost)
print("Difference:", best_cost - ort_cost)


Custom cost: 1820.2902196603723
OR-Tools cost: 1265.341
Difference: 554.9492196603724
